# Helpers

This notebook provides functions for the exercise notebook. It is not necessary to read through.

In [ ]:
import matplotlib.pyplot as plt
import numpy.random as nr

from numpy import pi as PI, cos, sin

%matplotlib inline

## Simple toy model

In [ ]:
binsize = 0.001  # seconds

def link(z):
    return np.exp(z)

def toyModel(X, theta):
    # compute the natural rate, the firing rate and the firing probability
    z = np.dot(theta, X)    # natural rate (a.u.)
    r = link(z) / binsize   # firing rate (Hz)
    p = r * binsize         # instantaneous firing rate

    y = np.random.poisson(p)  # spikes
    return y, p


In [ ]:
from IPython.display import display, Markdown, Latex

def toygrad():
    return display(Markdown(r'$\begin{align} \frac{\partial}{\partial \theta} \log \mathcal{L} &= \sum_{k=1}^K \sum_{t=1}^T  \bigg( y_{kt} - \frac{\mathrm{e}^{z_{kt}}}{1 + \mathrm{e}^{z_{kt}}} \bigg) \mathbf{x}_{kt} = \mathbf{r}^\top \mathbf{X},  \end{align}$ for residual error $\mathbf{r}_{kT+t} = y_{kt} - \frac{1}{1 + \mathrm{e}^{-z_{kt}}}$ and design matrix with rows $\mathbf{X}_{[kT+t,\dot{}]} = \mathbf{x}_{kt}$. '))

def prgrad():
    return display(Markdown(r'''$\begin{align} \frac{\partial}{\partial \theta} \log p(\theta)  &= - \left[0,\ \beta^\top \Sigma_\theta^{-1}, \ \psi ^\top \Sigma_\psi^{-1}\right]^\top + const. \end{align}$ 
    
Note: we can also stick to $\theta$, not breaking it up into $b$, $\beta, \psi$, if we define $P_\theta = \Sigma_\theta^{-1}$ as 
$\mathbf{P}_\theta = \begin{pmatrix}    
                \mathbf{0} & \mathbf{0} & \mathbf{0}
                \\            
                \mathbf{0} & \boldsymbol{\Sigma_\phi}^{-1} & \mathbf{0}
                \\
                \mathbf{0} & \mathbf{0} & \boldsymbol{\Sigma_\psi}^{-1}
              \end{pmatrix}$.              
Note the zero precision for bias $b$, for which we did not assume a prior. Then,

$ \frac{\partial}{\partial \theta} \log p(\theta)  = - \theta^\top P_\theta + const. $'''))

def toyll():
    return display(Markdown(r'''```python
def ll(theta):
    z = np.dot(theta, X)
    return np.sum( y * z - link(z) )```'''))

def toydll():
    return display(Markdown(r'''```python
def dll(theta):
    z = np.dot(theta, X)
    r = y - link(z)
    return np.dot(X, r)```'''))

def toymap():
    return display(Markdown(r'''```python
def po(theta):
    return ll(theta) + pr(theta)

def pr(theta):
    beta, psi = theta[1:delta+1], theta[delta+1:]
    lpr  = - .5 * (  np.dot(np.dot(beta,P_beta), beta) 
                   + np.dot(np.dot(psi, P_psi ), psi) )        
    return lpr```'''))

def toydmap():
    return display(Markdown(r'''```python
def dpo(theta):
    return dll(theta) + dpr(theta)

def dpr(theta):
    beta, psi = theta[1:delta+1], theta[delta+1:]
    return - np.hstack((0., np.dot(beta, P_beta), np.dot(psi, P_psi))) # leading zero for d/db```'''))

def toydme():
    return display(Markdown(r'''```python
def extendedDesignMatrix(y): 
    nbins = y.shape[0]
    t = np.arange(nbins) * binsize  # time axis

    X = np.zeros((1 + delta + tau, nbins))

    X[0,:] = 1.       # bias
    if delta > 0:
        X[1, :] = U   # instantaneous input
    for i in range(1,delta):
        X[i+1, i+1:] = U[:-(i+1)]

    for i in range(tau):
        X[delta+i+1,i+1:] = y[:-(i+1)]        

    return X```'''))

## Dataset for inference with extended model

In [ ]:
def createDataset(U, T, K, theta, delta):
    b, beta, psi = theta[0], theta[1:delta+1], theta[delta+1:]
    tau = len(psi)
    
    nbins = T*K
    X = toyDesignMatrix(U=U, T=T, K=K)
    z = np.dot(np.hstack((b, beta)), X)

    y = np.zeros(T*K, dtype=np.int64)                
    eps = np.random.uniform(size=z.shape)
    psi = psi[::-1]
    for k in range(K):
        for t in range(1,tau):        
            idx = k*T+t
            n = np.min((tau, t))
            h = y[idx-n : idx] 
            y[idx] = np.random.poisson( link(z[idx] + np.inner(h, psi[-n:])) )
        for t in range(tau,T):        
            idx = k*T+t
            h = y[idx-tau : idx] 
            y[idx] = np.random.poisson( link(z[idx] + np.inner(h, psi)) )
    
    return y    